In [1]:
import logging
import os

for log_file in [ "logger.log", "error.log"]:
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"Log file {log_file} are cleaned.")

logging.basicConfig(
    filename='logger.log',
    level=logging.DEBUG,
    format= '%(filename)s:%(lineno)s %(levelname)s: %(message)s'
)

print("Logging configured.")


Log file logger.log are cleaned.
Logging configured.


In [7]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.google_search_tool import google_search
from google.genai import types
from typing import List
from google.adk.runners import InMemoryRunner


In [3]:
retry_config = types.HttpRetryOptionsDict(
    attempts=5,
    exp_base= 7,
    initial_delay= 1,
    http_status_codes=[429,500,503,504]
)

In [4]:
def paper_count(papers: str):
    """  This function counts the number of papers in a list of strings.
    Args:
      papers: A list of strings, where each string is a research paper.
    Returns:
      The number of papers in the list."""
    
    return len(papers)

In [5]:
google_search_agent = LlmAgent(
    name="google_search_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options= retry_config),
    instruction="""Use the google_search tool to find information on the given topic. Return the raw search results.
    If the user asks for a list of papers, then give them the list of research papers you found and not the summary.""",
    tools=[google_search]
)

In [6]:
root_agent = LlmAgent(
    name= "root_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options= retry_config),
    instruction="""Your task is to find research papers and count them. 

    You MUST ALWAYS follow these steps:
    1) Find research papers on the user provided topic using the 'google_search_agent'. 
    2) Then, pass the papers to 'count_papers' tool to count the number of papers returned.
    3) Return both the list of research papers and the total number of papers.""",
    tools=[paper_count, AgentTool(agent=google_search_agent)]
)

In [8]:
runner = InMemoryRunner(agent= root_agent)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "D:\Google Course\myenv\Lib\site-packages\google\adk\agents", which implies app name "agents".


In [14]:
response = await runner.run_debug(
    "what are the research on alopecia areta?"
)

print(response)


 ### Created new session: debug_session_id

User > what are the research on alopecia areta?


App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "D:\Google Course\myenv\Lib\site-packages\google\adk\agents", which implies app name "agents".


root_agent > The research on alopecia areata has significantly advanced, focusing on understanding its causes and developing targeted therapies. Key areas include:

*   **Immunological Pathways:** Alopecia areata is an autoimmune condition where the immune system attacks hair follicles. Research is investigating specific immune cells and pathways to rebalance the immune response.
*   **JAK Inhibitors:** These drugs target specific immune signaling pathways to reduce inflammation and have shown promise in treating severe cases.
*   **Novel Treatment Approaches:** This includes microneedle patches for targeted delivery of immune-regulating molecules and platelet-rich plasma (PRP) therapy.
*   **Genetic and Environmental Factors:** Studies are identifying genetic predispositions and environmental factors contributing to the condition.
*   **Psychological Impact:** Research also addresses the significant psychological distress, such as anxiety and depression, associated with alopecia areat

C:\Users\User\AppData\Local\Temp\ipykernel_8088\1159781456.py:1: RuntimeWarning: coroutine 'Runner.run_debug' was never awaited
  response = await runner.run_debug(
